In [1]:
from wobbles.workflow.integrate_single_orbit import integrate_orbit
from wobbles.workflow.compute_distribution_function import compute_df
from wobbles.disc import Disc
from wobbles.potential_extension import PotentialExtension
from wobbles.distribution_function import DistributionFunction
import numpy as np

import galpy
from galpy import util
from galpy.potential import MovingObjectPotential
from galpy.potential import MWPotential2014
from galpy.potential import evaluatePotentials
from galpy.util.bovy_conversion import get_physical
from galpy.orbit import Orbit

import astropy.units as apu
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator

## First do the computation with wobbles

In [2]:
f = open('MW14pot_100', "rb")
potential_extension = pickle.load(f)
units = potential_extension.units
f.close()

# Initialize Sagitarrius orbit
t_orbit = -1.64 # Gyr
N_tsteps = 1200
time_Gyr = np.linspace(0., t_orbit, N_tsteps) * apu.Gyr 
orbit_init_sag = [283. * apu.deg, -30. * apu.deg, 26. * apu.kpc,
        -2.6 * apu.mas/apu.yr, -1.3 * apu.mas/apu.yr, 140. * apu.km/apu.s] # Initial conditions of the satellite
sag_orbit_phsical_off = integrate_orbit(orbit_init_sag, potential_extension.galactic_potential, time_Gyr)
satellite_orbit_list = [sag_orbit_phsical_off]

# endow satellite with mass
satellite_potential_1 = galpy.potential.HernquistPotential(amp=2.*1e10*apu.M_sun,a= 3.*apu.kpc) 
satellite_potential_2 = galpy.potential.HernquistPotential(amp=2.*0.2e9*apu.M_sun,a=0.65*apu.kpc) 
satellite_potential = satellite_potential_1 + satellite_potential_2
galpy.potential.turn_physical_off(satellite_potential)
satellite_potential_list = [satellite_potential]

# Initialize the main class
disc = Disc(potential_extension, potential_extension)
time_internal_units = sag_orbit_phsical_off.time()
velocity_dispersion = [20.5]
normalization = [1.]

distribution_function, delta_J, force = compute_df(disc, time_internal_units,
                          satellite_orbit_list, satellite_potential_list, velocity_dispersion, normalization, 
                                                   verbose=False)

dummy_potential = galpy.potential.HernquistPotential(amp=0.*1e10*apu.M_sun,a= 3.*apu.kpc)  
distribution_function_no_perturbation, _, _ = compute_df(disc, time_internal_units,
                          satellite_orbit_list, [dummy_potential], velocity_dispersion, normalization, 
                                                     verbose=False)

## Plot results

In [3]:
fig = plt.figure(1)
fig.set_size_inches(20, 6)
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

_z = np.linspace(0, 2, len(distribution_function.A))
ax1.plot(_z, distribution_function.A)
ax1.plot(_z, distribution_function_no_perturbation.A)
ax1.set_xlabel('z [kpc]', fontsize=16)
ax1.set_ylabel('asymmetry', fontsize=16)

_z = np.linspace(-2, 2, len(distribution_function.mean_v_relative))
ax2.plot(_z, distribution_function.mean_v_relative, label='with perturbation')
ax2.plot(_z, distribution_function_no_perturbation.mean_v_relative, label='steady state')
ax2.set_xlabel('z [kpc]', fontsize=16)
ax2.set_ylabel(r'$\langle v_z \rangle$', fontsize=16)
ax2.legend(fontsize=18, frameon=False)

## Now we'll do the full computation by integrating the orbits of a bunch of test particles in the Milky Way + satellite potential. 

### We'll initialize the vertical positions and velocities using the unperturbed solution

In [4]:
class InterpolatedPDF(object):

    def __init__(self, pdf, param_ranges):
        
        """
        This class takes a 2 dimension probability distribution (pdf) and samples from it. Samples are 
        drawn between (param_ranges[0][0], param_ranges[0][1]) for the first parameter, and 
        (param_ranges[1][0], param_ranges[1][1]) for the second parameter.
        """
        
        # normalize to one for rejection sampling
        norm = np.max(pdf)
        self._pdf = pdf/norm
        
        nbins = np.shape(self._pdf)[0]
        points = []
        for prange in param_ranges:
            points.append(np.linspace(prange[0], prange[-1], nbins))
        self.param_ranges = param_ranges
        self.interp = RegularGridInterpolator(points, self._pdf)

    def sample(self, ndraw):
        
        """
        Sample the 2D distribution function with rejection sampling
        """
        points = np.empty((ndraw, 2))
        count = 0
        print('sampling... ')
        while count < ndraw:
            
            sample_dim_1 = np.random.uniform(self.param_ranges[0][0], self.param_ranges[0][1])
            sample_dim_2 = np.random.uniform(self.param_ranges[1][0], self.param_ranges[1][1])
            point = (sample_dim_1, sample_dim_2)
            
            # evaluate the PDF at the sampled coordinate
            p = self.interp(point)
            # accept the sample with probability p 
            if p >= np.random.rand():
                points[count, 0], points[count, 1] = point[0], point[1]
                count += 1
        return points
    
    def rebin_pdf(self, samples, nbins=50):
        """
        Rebin samples from the 2D distribution function
        """
        f_approx, _, _ = np.histogram2d(samples[:,0], samples[:,1], range=self.param_ranges, bins=nbins)
        return f_approx

f = distribution_function_no_perturbation.function
fig = plt.figure(1)
plt.imshow(f)
plt.annotate('steady state distribution function\n(SSDF)', xy=(0.05, 0.8), xycoords='axes fraction', color='w', 
            fontsize=12)
plt.show()

pranges = [[-2, 2], [-120, 120]]
interp_f = InterpolatedPDF(f, pranges)
n_samples = 50000
samples = interp_f.sample(n_samples)

z_positions, vz_samples = samples[:, 0], samples[:, 1]
f_approx = interp_f.rebin_pdf(samples, nbins=40)

plt.figure(2)
plt.imshow(f_approx)
plt.annotate('samples from SSDF', xy=(0.05, 0.9), xycoords='axes fraction', color='w', 
            fontsize=12)

plt.figure(3)
plt.hist(z_positions, bins=20, label='z positions')
plt.legend(fontsize=12, frameon=False)
plt.gca().set_xlabel('z [kpc]', fontsize=14)

plt.figure(4)
plt.hist(vz_samples, bins=20, label='vertical velocities')
plt.legend(fontsize=12, frameon=False)
plt.gca().set_xlabel('v_z [km/sec]', fontsize=14)

### Now initialize a bunch of test particles with values of (z, v_z) sampled from the unperturbed distribution function

In [5]:
class TestParticle(object):
    
    def __init__(self, z, vz):
        
        self.z = z
        self.vz = vz
        self.orbit = None
        self.orbit_with_satellite = None
        
test_particles = []
for i in range(0, len(z_positions)):
    test_particles.append(TestParticle(z_positions[i], vz_samples[i]))

## Integrate the orbit of each star in the combined moving satellite potential and the potential of the MW

In [6]:
# the total potential from satellite + Milky Way
satellite_moving_potential = MovingObjectPotential(satellite_orbit_list[0], satellite_potential, 
                                                   ro=units['ro'], vo=units['vo'])

total_potential = potential_extension.galactic_potential
total_vertical_potential = galpy.potential.toVerticalPotential(total_potential, 1., phi=0.)
galpy.potential.turn_physical_off(total_vertical_potential)

total_potential_with_satellite = satellite_moving_potential + potential_extension.galactic_potential
total_vertical_potential_with_satellite = galpy.potential.toVerticalPotential(total_potential_with_satellite, 1., phi=0.)
galpy.potential.turn_physical_off(total_vertical_potential_with_satellite)

time_internal_units = time_internal_units[::-1]

ntotal = len(test_particles)
for i, particle in enumerate(test_particles):
    if i%1000 == 0:
        print('number remaining: ', ntotal - i)
    orbit_no_satellite = Orbit([particle.z/units['ro'], particle.vz/units['vo']], ro=units['ro'], vo=units['vo'])
    orbit_no_satellite.turn_physical_off()
    orbit_no_satellite.integrate(time_internal_units, total_vertical_potential)
    particle.orbit = orbit_no_satellite
    
    orbit_with_satellite = Orbit([particle.z/units['ro'], particle.vz/units['vo']], ro=units['ro'], vo=units['vo'])
    orbit_with_satellite.turn_physical_off()
    orbit_with_satellite.integrate(time_internal_units, total_vertical_potential_with_satellite)
    particle.orbit_with_satellite = orbit_with_satellite
   

### Now rebin the test particles in order to compute the asymmetry and mean vertical velocity

In [7]:
def compute_asymmetry(density, z, z_0, tol=1e-9):
    
    zs = 2. * z_0 - z
    zf = np.sort(np.append(z, zs))
    funcp = interp1d(z, density, kind='linear', fill_value='extrapolate')
    p = funcp(zf)
    A = (p - p[::-1]) / (p + p[::-1])
    zA = zf - z_0
    A = A[zA >= 0.]
    zA = zA[zA >=0]
    return zA, A

z_final = np.array([particle.orbit_with_satellite.x(0.) * units['ro'] for particle in test_particles])
vz_final = np.array([particle.orbit_with_satellite.vx(0.) * units['vo'] for particle in test_particles])

# rebin
nbins = 51
zbins = np.linspace(-2, 2, nbins)
vzbins = np.linspace(-20, 20, nbins)

density, _zbins = np.histogram(z_final, bins=zbins)
step = _zbins[1] - _zbins[0]
zbins = _zbins[0:-1] + step/2

fig = plt.figure(1)
plt.plot(zbins, density)

from wobbles.util import fit_sec_squared
(_, z_sun, _) = fit_sec_squared(density, zbins)
za, A = compute_asymmetry(density, zbins, z_sun)
fig = plt.figure(2)
plt.plot(za, A)